In [1]:
import numpy as np
import pandas as pd

/tmp/ipykernel_6023/1662815981.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

24/01/24 12:08:11 WARN Utils: Your hostname, venkatesh-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/01/24 12:08:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/01/24 12:08:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.version

'3.3.2'

In [4]:
df = spark.createDataFrame([(1, 'Alice'),(2, 'Bob'),(3, 'Cycle')], schema=['roll_no','name'])
df

DataFrame[roll_no: bigint, name: string]

In [5]:
df.show()

+-------+-----+
|roll_no| name|
+-------+-----+
|      1|Alice|
|      2|  Bob|
|      3|Cycle|
+-------+-----+



In [6]:
df.show(1)

+-------+-----+
|roll_no| name|
+-------+-----+
|      1|Alice|
+-------+-----+
only showing top 1 row



In [7]:
df.head()

Row(roll_no=1, name='Alice')

In [8]:
df.select('*').show()

+-------+-----+
|roll_no| name|
+-------+-----+
|      1|Alice|
|      2|  Bob|
|      3|Cycle|
+-------+-----+



In [9]:
df.columns

['roll_no', 'name']

In [10]:
df.select('name').show()

+-----+
| name|
+-----+
|Alice|
|  Bob|
|Cycle|
+-----+



In [11]:
df.printSchema()

root
 |-- roll_no: long (nullable = true)
 |-- name: string (nullable = true)



In [12]:
df.toPandas()

,roll_no,name
0,1,Alice
1,2,Bob
2,3,Cycle


In [13]:
type(df.toPandas())

pandas.core.frame.DataFrame

# Grouping

In [14]:
df = spark.createDataFrame([
    ['red', 'banana', 1, 10], ['blue', 'banana', 2, 20], ['red', 'carrot', 3, 30],
    ['blue', 'grape', 4, 40], ['red', 'carrot', 5, 50], ['black', 'carrot', 6, 60],
    ['red', 'banana', 7, 70], ['red', 'grape', 8, 80]], schema=['color', 'fruit', 'v1', 'v2'])
df.show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|  red|banana|  1| 10|
| blue|banana|  2| 20|
|  red|carrot|  3| 30|
| blue| grape|  4| 40|
|  red|carrot|  5| 50|
|black|carrot|  6| 60|
|  red|banana|  7| 70|
|  red| grape|  8| 80|
+-----+------+---+---+



In [16]:
df.groupby('fruit').count().show()

+------+-----+
| fruit|count|
+------+-----+
| grape|    2|
|banana|    3|
|carrot|    3|
+------+-----+



In [17]:
df.groupby('color').avg().show()

+-----+-------+-------+
|color|avg(v1)|avg(v2)|
+-----+-------+-------+
|  red|    4.8|   48.0|
| blue|    3.0|   30.0|
|black|    6.0|   60.0|
+-----+-------+-------+



# Working with SQL

In [19]:
df.createOrReplaceTempView('fruits_table')
spark.sql('select * from fruits_table').show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|  red|banana|  1| 10|
| blue|banana|  2| 20|
|  red|carrot|  3| 30|
| blue| grape|  4| 40|
|  red|carrot|  5| 50|
|black|carrot|  6| 60|
|  red|banana|  7| 70|
|  red| grape|  8| 80|
+-----+------+---+---+



In [20]:
spark.sql('select count(*) from fruits_table').show()

+--------+
|count(1)|
+--------+
|       8|
+--------+



In [22]:
spark.sql('select color,fruit from fruits_table').show()

+-----+------+
|color| fruit|
+-----+------+
|  red|banana|
| blue|banana|
|  red|carrot|
| blue| grape|
|  red|carrot|
|black|carrot|
|  red|banana|
|  red| grape|
+-----+------+

